In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec

2025-06-10 12:24:50.079939: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-10 12:24:50.084457: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 12:24:50.111959: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 12:24:50.181586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 12:24:50.282077: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [2]:
path= "../raw_data/train_df_ml_clean.csv"

In [3]:
clean_df = pd.read_csv(path)

In [4]:
small_clean_df = clean_df.sample(frac=0.1)
len(small_clean_df)

360000

In [5]:
small_clean_df.head()

,label,text,clean_text
1636754,0,Duplicate as last DVD collection. DON'T WASTE ...,duplicate a last dvd collection dont waste you...
1472222,1,"loved this movie: one more sequel, thats what ...",loved this movie one more sequel thats what th...
1889937,1,The Best of the best Of Waylon Jennings: Waylo...,the best of the best of waylon jennings waylon...
3050569,0,Story seemed to be rushed.: Typically a fan of...,story seemed to be rushed typically a fan of t...
1520899,1,Pooh's Grand Adventure: My children (2 and 3 y...,poohs grand adventure my child and yearold gir...


In [6]:
X_small = small_clean_df['clean_text'].to_numpy()
y_small = small_clean_df['label'].to_numpy()

X_train_small, X_val_small, y_train_small, y_val_small = train_test_split(
    X_small, y_small, test_size=0.2, random_state=42
)

In [7]:
X_val_small[:10]

array(['problem fixed after reading the first two negative review i delayed for month getting this battery grip until i found out canon came out with a fix around note the two negative review are dated google canon bge2 service problemi shot about two thousand photo on a cold thanksgiving morning with a monopod attached to a canon 200mm f2 l lens the performance of the battery grip wa solid never had a problem now the battery ha become a permanent fixture on my 30di gave it star because at it overpriced i wish there wa a aftermarket battery grip like the opteka for the rebel i see that jenis make one but couldnt seem to figure out how to buy it',
       'wandering and remembering great listening of course i am a newell oler fan i dont believe he could put out a bad cd',
       'not that great this pillow is not a good a others i have slept on i wish i would have just gone to a store and picked one out to me it just seems like a different type of foam the shape is nice but it doesnt do 

numpy.ndarray

In [8]:
import matplotlib.pyplot as plt

In [12]:
length_array = [len(seq) for seq in X_train_small.astype(str)]
length_array

[136,
 159,
 214,
 573,
 813,
 187,
 665,
 132,
 512,
 581,
 890,
 609,
 365,
 449,
 662,
 399,
 500,
 196,
 438,
 281,
 788,
 266,
 623,
 198,
 443,
 203,
 603,
 415,
 159,
 673,
 156,
 220,
 192,
 345,
 261,
 523,
 260,
 145,
 666,
 84,
 157,
 464,
 158,
 981,
 486,
 549,
 182,
 197,
 282,
 198,
 497,
 411,
 143,
 113,
 124,
 436,
 198,
 319,
 505,
 250,
 363,
 129,
 278,
 324,
 608,
 163,
 427,
 163,
 230,
 271,
 228,
 221,
 422,
 449,
 778,
 189,
 359,
 793,
 540,
 596,
 245,
 168,
 347,
 179,
 437,
 204,
 490,
 197,
 420,
 152,
 433,
 300,
 600,
 189,
 709,
 578,
 584,
 949,
 420,
 130,
 166,
 471,
 103,
 739,
 200,
 315,
 237,
 121,
 676,
 118,
 207,
 213,
 783,
 152,
 135,
 782,
 188,
 97,
 262,
 348,
 203,
 369,
 492,
 382,
 263,
 128,
 764,
 362,
 556,
 430,
 121,
 786,
 461,
 155,
 234,
 416,
 934,
 242,
 293,
 217,
 284,
 398,
 612,
 906,
 385,
 254,
 274,
 326,
 227,
 569,
 377,
 401,
 815,
 604,
 148,
 886,
 120,
 477,
 468,
 244,
 295,
 261,
 947,
 109,
 706,
 212,
 254,


In [10]:
plt.hist(length_array)

NameError: name 'length_array' is not defined

In [14]:
X_train_token_s = [text_to_word_sequence(_) for _ in X_train_small.astype(str)]
X_val_token_s = [text_to_word_sequence(_) for _ in X_val_small.astype(str)]

In [19]:
# Learn embedding representation of words in reviews
word2vec = Word2Vec(sentences=X_train_token_s)
# Store words and trained embeddings in wv
wv = word2vec.wv

In [17]:
# Length of word embed
X_train_small[:10]

array(['a great read the author take you on a long journey through the vastness of russia over many century compelling wellwritten and education',
       'dr seuss sleep book i used to read this book a a child when you read it you actually start yawning and cant stop it is a classic and fun to read before bedtime',
       'i love my new dish i got my order on time and they were in perfect condition this is my first pfaltzgraff purchase the price is affordable the best thing is you can collect additional piece to go with a starter set',
       'shallow gal tender murderer woman who kill by trina robbins is a puff piece twenty error filled chapter written with a wink and a nudge robbins couldnt of picked easier murderer to write about and they have all been written about before with more detail her section on dorothea puente the sacramento rooming house murderer wa almost fiction she also left out ton of information on the bender this book is fine if you are a novice but anyone who ha re

In [23]:
wv.most_similar("age", topn=10)

[('adult', 0.5586798787117004),
 ('youth', 0.5298827886581421),
 ('teen', 0.5279924273490906),
 ('preteen', 0.5142388343811035),
 ('child', 0.5030690431594849),
 ('teenager', 0.48494648933410645),
 ('young', 0.47330784797668457),
 ('kid', 0.4708179235458374),
 ('parent', 0.46737968921661377),
 ('generation', 0.45931896567344666)]

In [24]:
len(wv.key_to_index.keys())

52254